In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code/

/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code


In [3]:
!pip install pytorchcv -q
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from utils.datasets.fer2013_ds import FERDataset
# from models.vgg16_cbam import  VGG19_CBAM
from models.resnet_cbam import ResidualNet , cbam_resnet50
from models.vggnet import vgg16_bn, vgg19, vgg19_bn, vgg16
from models.resnet import resnet50, resnet34
from models.vggnet_cbam import vgg16_cbam, vgg19_cbam
from models.vggnet_cbam_pre import *
from models.ResNetVDSR import resnetvdsr_dropout1
from models.resmasking import *
from models.BamNetwork import *
from models.CANetwork import *
from models.New_model import *
from models.RIRAttention import *
from torch.utils.data import DataLoader

from utils.visualize.show_img import show_image_dataset
from trainer.fer2013_trainer import FER2013_Trainer
import json

import matplotlib.pyplot as plt 
import cv2
import numpy as np

In [4]:
# config_path = "/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code/configs/config_fer2013.json"
# configs = json.load(open(config_path))

# train_loader = FERDataset( "train", configs)
# val_loader = FERDataset("val", configs)
# test_loader_ttau = FERDataset("test", configs, ttau = True, len_tta = 48) 
# test_loader = FERDataset("test", configs, ttau = False, len_tta = 48) 

# train_ds = DataLoader(train_loader,batch_size=configs["batch_size"],num_workers=configs["num_workers"],
#                   pin_memory=True, shuffle=True)
# val_ds = DataLoader(val_loader, batch_size = configs["batch_size"], num_workers=configs["num_workers"],
#                 pin_memory=True, shuffle=False)
# test_ds = DataLoader(test_loader, batch_size= 1,num_workers=configs["num_workers"],
#                   pin_memory=True, shuffle=False)

In [5]:
# dataiter = iter(train_ds)
# images, labels = dataiter.next()

In [6]:
#  model = vgg16()
# model = VGGBN16CbamMultiFc()
# model = vgg16_bn_test()
# model = ResidualNet("ImageNet", 18, 7,"Cbam")
# model = vgg16_bn_cbam_pre(7)

# model = MultiFCVGGnetCBam(model)
# model = resnet34(num_classes = 7)
model = MyModel_dropout1()
# model = CAModel_dropout1()
# model = resmasking_dropout1()
# model = vgg19_bn()

In [7]:
# print(model)

In [8]:
from torchsummary import summary

summary(model, ((3,224,224)), batch_size = 32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 112, 112]           9,408
       BatchNorm2d-2         [32, 64, 112, 112]             128
              ReLU-3         [32, 64, 112, 112]               0
         MaxPool2d-4           [32, 64, 56, 56]               0
            Conv2d-5           [32, 64, 56, 56]          36,864
       BatchNorm2d-6           [32, 64, 56, 56]             128
              ReLU-7           [32, 64, 56, 56]               0
            Conv2d-8           [32, 64, 56, 56]          36,864
       BatchNorm2d-9           [32, 64, 56, 56]             128
             ReLU-10           [32, 64, 56, 56]               0
       BasicBlock-11           [32, 64, 56, 56]               0
           Conv2d-12           [32, 64, 56, 56]          36,864
      BatchNorm2d-13           [32, 64, 56, 56]             128
             ReLU-14           [32, 64,

In [9]:
..
# import torch
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

SyntaxError: ignored

In [ ]:
a = torch.tensor(1.0)
print(torch.sigmoid(a))
# import torchvision
# images, labels = next(iter(train_ds))
# grid = torchvision.utils.make_grid(images)
# writer.add_image('images', grid, 0)
# writer.add_graph(model, images)
# writer.close()

In [ ]:
# !pip install tensorboard
# %load_ext tensorboard
# %tensorboard --logdir runs

In [ ]:
# import torch
# t = torch.tensor([[[1, 2],
#                    [3, 4]],
#                    [[5, 6],
#                   [7, 8]]])
# x =  torch.flatten(t)

# print(len(x))

In [ ]:
# !git clone https://github.com/phamquiluan/ResidualMaskingNetwork.git

In [ ]:
# !pip install -r /content/ResidualMaskingNetwork/requirements.txt

In [ ]:
# !python /content/ResidualMaskingNetwork/main_fer2013.py

In [ ]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None
        

        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
        #     elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
        #         nn.init.constant_(m.weight, 1)
        #         nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class ChannelAttention(nn.Module):
    def __init__(self, in_channels, ratio = 16):
        super(ChannelAttention,self).__init__()

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.conv1 = nn.Conv2d(in_channels, in_channels // ratio, 1, bias = False)
        self.relu = nn.PReLU(in_channels // ratio)
        self.conv2 = nn.Conv2d(in_channels // ratio, in_channels, 1, bias = False)

    def forward(self, x):
        avg_out = self.conv2(self.relu(self.conv1(self.avg_pool(x))))
        # max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out
        scale = torch.sigmoid(out)
        return  x * scale


class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = torch.sigmoid(x_out) # broadcasting
        return x * scale


class ModuleAttention(nn.Module):
    def __init__(self, in_channels, ratio ):
        super(ModuleAttention, self).__init__()

        self.ca = ChannelAttention(in_channels, ratio = ratio)
        self.sa = SpatialGate()
    def forward(self, x):
        out = self.ca(x)
        out = self.sa(out)
        return out


In [ ]:
"""
Create block RIR BAM
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
# from .resnet import BasicBlock, Bottleneck
import math
from math import sqrt
# from .Bam import *


def conv2d(in_channels, out_channels, kernel_size, bias = True):
    return nn.Conv2d(in_channels, out_channels, kernel_size, padding = (kernel_size//2), bias = bias)

def conv1x1(in_channels, out_channels, stride = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = stride, bias = False)

class DownsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = 2 , padding = 0, bias = True)
        self.pool = nn.MaxPool2d(2)
        self.bn = nn.BatchNorm2d(out_channels, eps=1e-3)
    def forward(self, x):
        out = torch.cat([self.conv(x), self.pool(x)], 1)
        out = self.bn(x)
        return F.relu(out)

class Upsampler(nn.Sequential):
    def __init__(self, conv, scale, channels, bn = False, act = False, bias = True):
        
        layers = []
        if (scale & (scale - 1)) == 0:
            for _ in range(int(math.log(scale, 2))):
                layers.append(conv(channels, 4* channels, 3, bias))
                layers.append(nn.PixelShuffle(2))
                if bn:
                    layers.append(nn.BatchNorm2d(channels))
                if act:
                    layers.append(act())
        elif scale == 3:
            layers.append(conv(channels, 9 * channels, 3 , bias))
            layers.append(nn.PixelShuffle(3))
            if bn:
                layers.append(BatchNorm2d(channels))
            if act:
                layers.append(act())
        else:
            raise NotImplementedError

        super(Upsampler, self).__init__(*layers)

class ResBlockAttention(nn.Module):
    def __init__(self, conv, channels, kernel_size, reduction, batch_norm = False):
        super(ResBlockAttention, self).__init__()
        ResLayers = []
        
        self.conv1 = conv(channels, channels, kernel_size, bias = False)
        self.bn = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace= True)
        # self.conv2 = conv(channels, channels, kernel_size, bias = False)
        # self.bn2 = nn.BatchNorm2d(channels)
        # self.relu = nn.ReLU(inplace= True)

        ResLayers.append(ModuleAttention(channels, ratio = reduction))
        self.layers = nn.Sequential(*ResLayers)

    def forward(self, x):
        out = self.layers(x)
        out = self.conv1(out)
        out = self.bn(out)
        out += x
        out = self.relu(x)
        
        return out

class ResGroup(nn.Module):
    def __init__(self, conv, channels,kernel_size,reduction, n_resblocks):
        super(ResGroup, self).__init__()
        
        ResLayers = []
        ResLayers = [
            ResBlockAttention(conv, channels, kernel_size, reduction)
            for _ in range(n_resblocks)
        ]
        
        self.layers = nn.Sequential(*ResLayers)

    def forward(self, x):
        out = self.layers(x)

        out += x

        return out

class Module1(nn.Module):
    def __init__(self, channels, n_resgroups, n_resblocks, scale, conv = conv2d):
        super(Module1, self).__init__()
        
        kernel_size = 3
        reduction = 16
        size1 = (56,56)
        size2 = (28,28)
        size3 = (14,14)

        self.ModuleAttention = ModuleAttention(channels, ratio = reduction)
 
        self.down1 = nn.MaxPool2d(2)
        self.down2 = nn.MaxPool2d(2)
        self.down3 = nn.MaxPool2d(2)

        groupLayers = [
            ResGroup(conv, channels, kernel_size, reduction = reduction, n_resblocks = n_resblocks )
            for _ in range(n_resgroups)
        ]

        self.deepblock = nn.Sequential(*groupLayers)

        self.up1 = nn.UpsamplingBilinear2d(size = size1)
        self.up2 = nn.UpsamplingBilinear2d(size = size2)
        self.up3 = nn.UpsamplingBilinear2d(size = size3)

        self.out = nn.Sequential(
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(channels, channels, kernel_size=1, stride =1, bias = False),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(channels, channels, kernel_size=1, stride = 1, bias = False),
            nn.Sigmoid()
        )
    def forward(self, x):
        out_cat = self.ModuleAttention(x)
        
        out = self.down1(x)  #28
        out_deep1 = self.deepblock(out)
        out = self.down2(out_deep1) #14
        out_deep2 = self.deepblock(out) 
        out = self.down3(out_deep2)
        out_deep3 = self.deepblock(out)

        out = self.up3(out_deep3)
        out = out + out_deep2
        out = self.up2(out_deep2) #28
        out = out + out_deep1
        out = self.up1(out) # 56
        out = out 

        out = self.out(out)
        print(out.size())
        output = (1+out)*out_cat

        return output

class Module2(nn.Module):
    def __init__(self, channels, n_resgroups, n_resblocks, scale, conv = conv2d):
        super(Module2, self).__init__()
        
        kernel_size = 3
        reduction = 16
        size1 = (28,28)
        size2 = (14,14)
        size3 = (7,7)

        self.ModuleAttention = ModuleAttention(channels, ratio = reduction)
 
        self.down1 = nn.MaxPool2d(2)
        self.down2 = nn.MaxPool2d(2)
        self.down3 = nn.MaxPool2d(2)

        groupLayers = [
            ResGroup(conv, channels, kernel_size, reduction = reduction, n_resblocks = n_resblocks )
            for _ in range(n_resgroups)
        ]

        self.deepblock = nn.Sequential(*groupLayers)

        self.up1 = nn.UpsamplingBilinear2d(size = size1)
        self.up2 = nn.UpsamplingBilinear2d(size = size2)
        self.up3 = nn.UpsamplingBilinear2d(size = size3)

        self.out = nn.Sequential(
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(channels, channels, kernel_size=1, stride =1, bias = False),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(channels, channels, kernel_size=1, stride = 1, bias = False),
            nn.Sigmoid()
        )
    def forward(self, x):
        out_cat = self.ModuleAttention(x)
        
        out = self.down1(x)  #28
        out_deep1 = self.deepblock(out)
        out = self.down2(out_deep1) #14
        out_deep2 = self.deepblock(out) 
        out = self.down3(out_deep2)
        out_deep3 = self.deepblock(out)

        out = self.up3(out_deep3)
        out = out + out_deep2
        out = self.up2(out_deep2) #28
        out = out + out_deep1
        out = self.up1(out) # 56
        out = out 

        out = self.out(out)
        print(out.size())
        output = (1+out)*out_cat

        return output

class Module3(nn.Module):
    def __init__(self, channels, n_resgroups, n_resblocks, scale, conv = conv2d):
        super(Module3, self).__init__()
        
        kernel_size = 3
        reduction = 16
        size1 = (14,14)
        size2 = (7,7)

        self.ModuleAttention = ModuleAttention(channels, ratio = reduction)
 
        self.down1 = nn.MaxPool2d(2)
        self.down2 = nn.MaxPool2d(2)

        groupLayers = [
            ResGroup(conv, channels, kernel_size, reduction = reduction, n_resblocks = n_resblocks )
            for _ in range(n_resgroups)
        ]

        self.deepblock = nn.Sequential(*groupLayers)

        self.up1 = nn.UpsamplingBilinear2d(size = size1)
        self.up2 = nn.UpsamplingBilinear2d(size = size2)

        self.out = nn.Sequential(
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=1, stride =1, bias = False),
            nn.Sigmoid()
        )
    def forward(self, x):
        out_cat = self.ModuleAttention(x)

        out = self.down1(x)
        out_deep1 = self.deepblock(out)
        out = self.down2(out_deep1)
        out_deep2 = self.deepblock(out)

        out = self.up2(out_deep2)
        out = out + out_deep1
        out = self.up1(out)

        out = self.out(out)
        output = (1+out)*out_cat

        return output

class Module4(nn.Module):
    def __init__(self, channels, n_resgroups, n_resblocks, scale, conv = conv2d):
        super(Module4, self).__init__()
        
        kernel_size = 3
        reduction = 16
        # size1 = (14,14)
        # size2 = (7,7)

        self.ModuleAttention = ModuleAttention(channels, ratio = reduction)
 
        # self.down1 = nn.MaxPool2d(2)
        # self.down2 = nn.MaxPool2d(2)

        groupLayers = [
            ResGroup(conv, channels, kernel_size, reduction = reduction, n_resblocks = n_resblocks )
            for _ in range(n_resgroups)
        ]

        self.deepblock = nn.Sequential(*groupLayers)

        # self.up1 = nn.UpsamplingBilinear2d(size = size1)
        # self.up2 = nn.UpsamplingBilinear2d(size = size2)

        self.out = nn.Sequential(
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=1, stride =1, bias = False),
            nn.Sigmoid()
        )
    def forward(self, x):
        out_cat = self.ModuleAttention(x)
        out = self.deepblock(x)

        out = self.out(out)
        output = (1+out)*out_cat

        return output


In [ ]:
import traceback
import torch
import torch.nn as nn
import torch.nn.functional as F

from models.resnet import BasicBlock, Bottleneck, ResNet, resnet18

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

model_urls = {
    "resnet18": "https://download.pytorch.org/models/resnet18-5c106cde.pth",
    "resnet34": "https://download.pytorch.org/models/resnet34-333f7ec4.pth",
    "resnet50": "https://download.pytorch.org/models/resnet50-19c8e357.pth",
}

# from models.BamRcan import *

class BamNet1(ResNet):
    def __init__(self):
        super(BamNet1, self).__init__(
           block =  BasicBlock, layers = [3, 4, 6, 3], in_channels = 3,num_classes = 1000
           )
        

        state_dict = load_state_dict_from_url(model_urls['resnet34'], progress=True)
        self.load_state_dict(state_dict)

        self.fc = nn.Linear(512, 7)

        nn.init.kaiming_normal_(self.fc.weight)

        self.Myblock1 = Module1(64,3,6,2) # 2-4
        self.Myblock2 = Module2(128,3,6,2) # 3 -6
        self.Myblock3 = Module3(256,3,6,2) # 4- 8
        self.Myblock4 = Module4(512,3,6, 2) # 4 - 8
    

    def forward(self, x):  # 224
        x = self.conv1(x)  # 112
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)  # 56

        x = self.layer1(x)  # 56 :8
        x = self.Myblock1(x)
        # x = x * (1 + m)
        # x = x * m

        x = self.layer2(x)  # 28 : 4 = 7
        x = self.Myblock2(x)
        # x = x * (1 + m)
        # x = x * m

        x = self.layer3(x)  # 14 : 2 = 7
        x = self.Myblock3(x)
        # x = x * (1 + m)
        # x = x * m

        x = self.layer4(x)  # 7
        x = self.Myblock4(x)
        # x = x * (1 + m)
        # x = x * m

        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.fc(x)
        return x


def BamModel_dropout2(in_channels=3, num_classes=7):
    model = BamNet1()
    model.fc = nn.Sequential(
        nn.Dropout(0.4),
        nn.Linear(512, 7)
        # nn.Linear(512, num_classes)
    )
    return model

In [ ]:
model = BamModel_dropout2()

In [ ]:
from torchsummary import summary

summary(model, ((3,224,224)), batch_size = 32)